In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [6]:
from spark_PCA import *
from row_parser import *

In [15]:
# %pylab inline

import pandas as pd
import datetime as dt

from scipy.io import loadmat, savemat, whosmat

from collections import Counter
import re
import numpy as np
from numpy import shape

from glob import glob
from time import time
import sys
from row_parser import *
from spark_PCA import *
from pyspark.sql import DataFrame

In [20]:
!ls -l ./../data/DSE230_version_Data/whale_data_15mb.np

-rw-r--r--  1 Jim  staff  15696962 Dec  6 22:32 ./../data/DSE230_version_Data/whale_data_15mb.np


In [22]:
file_path = './../data/DSE230_version_Data/whale_data_15mb.np'
file_data = np.load(file_path)

In [26]:
file_data

array([[datetime.datetime(2010, 8, 10, 11, 45, 36, 626459), u'Cuviers',
        u'DT', ..., 0, 0, 0],
       [datetime.datetime(2010, 8, 10, 11, 46, 6, 4604), u'Cuviers',
        u'DT', ..., 0, 0, 0],
       [datetime.datetime(2010, 8, 10, 12, 11, 25, 866437), u'Cuviers',
        u'DT', ..., 0, 0, 0],
       ...,
       [datetime.datetime(2013, 4, 30, 7, 19, 41, 228925), u'Gervais',
        u'MC', ..., 0, 0, 0],
       [datetime.datetime(2013, 5, 1, 11, 37, 0, 617731), u'Gervais',
        u'MC', ..., 0, 0, 0],
       [datetime.datetime(2013, 5, 1, 11, 38, 46, 580242), u'Gervais',
        u'MC', ..., 0, 0, 0]], dtype=object)

In [24]:
schema = StructType([
     StructField('time', TimestampType(), True),
     StructField('species', StringType(), True),
     StructField('site', StringType(), True),
     StructField('rec_no', StringType(), True),
     StructField('bout_i', LongType(), True),
     StructField('peak2peak', DoubleType(), True),
     StructField('MSN', BinaryType(), True),
     StructField('MSP', BinaryType(), True),
     StructField('TPWS1', LongType(), True),
     StructField('MD1', LongType(), True),
     StructField('FD1', LongType(), True),
     StructField('TPWS2', LongType(), True),
     StructField('MD2', LongType(), True),
     StructField('FD2', LongType(), True),
     StructField('TPWS3', LongType(), True),
     StructField('MD3', LongType(), True),
     StructField('FD3', LongType(), True)
])

In [42]:
schema = {'time': np.datetime64,
           'species': 'object',
           'site': 'object',
           'rec_no': 'object',
           'bout_i': 'int64',
           'peak2peak': 'float64',
           'MSN': 'bool',
           'MSP': 'bool',
           'TPWS1': 'int64',
           'MD1': 'int64',
           'FD1': 'int64',
           'TPWS2': 'int64',
           'MD2': 'int64',
           'FD2': 'int64',
           'TPWS3': 'int64',
           'MD3': 'int64',
           'FD3': 'int64'}

In [43]:
df = pd.DataFrame(data=file_data)

In [47]:
df = pd.DataFrame(data=file_data, 
                  columns=['time', 'species', 'site', 'rec_no', 'bout_i', 'peak2peak', 'MSN', 'MSP', 'TPWS1', 'MD1', 'FD1', 'TPWS2', 'MD2', 'FD2', 'TPWS3', 'MD3', 'FD3'])

In [49]:
df = df.astype(dtype=schema)

In [51]:
df[:3]

,time,species,site,rec_no,bout_i,peak2peak,MSN,MSP,TPWS1,MD1,FD1,TPWS2,MD2,FD2,TPWS3,MD3,FD3
0,2010-08-10 11:45:36.626459,Cuviers,DT,01,5,50.2422,True,True,1,0,0,1,0,0,0,0,0
1,2010-08-10 11:46:06.004604,Cuviers,DT,01,5,55.0377,True,True,1,0,0,1,0,0,0,0,0
2,2010-08-10 12:11:25.866437,Cuviers,DT,01,5,44.9439,True,True,1,0,0,1,0,0,0,0,0


In [50]:
df.dtypes

time         datetime64[ns]
species              object
site                 object
rec_no               object
bout_i                int64
peak2peak           float64
MSN                    bool
MSP                    bool
TPWS1                 int64
MD1                   int64
FD1                   int64
TPWS2                 int64
MD2                   int64
FD2                   int64
TPWS3                 int64
MD3                   int64
FD3                   int64
dtype: object

In [55]:
df.groupby(['species']).count()

,time,site,rec_no,bout_i,peak2peak,MSN,MSP,TPWS1,MD1,FD1,TPWS2,MD2,FD2,TPWS3,MD3,FD3
species,,,,,,,,,,,,,,,,
Cuviers,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238
Gervais,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937,1937
